In [1]:
import torch, torch.nn as nn, torch.nn.functional as F, numpy as np

In [2]:
from torch.autograd import Variable

In [3]:
state_size = 24
comm_size = 128

communicate = nn.Sequential(nn.Linear(state_size, 32), nn.ReLU(), nn.Linear(32, comm_size))

# communication -> V(s, c)
other_agents_value = nn.Sequential(nn.Linear(comm_size, 64), nn.ReLU(), nn.Linear(64, 1))

In [7]:
class agent:
    state = Variable(torch.randn(1, state_size))
    comm = communicate(state)

class env: 
    comm = agent.comm.data.numpy()
    
class recipient:
    comm = Variable(torch.from_numpy(env.comm), requires_grad=True)
    
    value = other_agents_value(comm)
    
    comm_loss = -value.mean()
    
    dloss_dcomm = torch.autograd.grad(comm_loss, [comm])[0]
    
class agent_training:
    agent.comm.backward(gradient=recipient.dloss_dcomm)
    

In [36]:
next(other_agents_value.parameters()).grad

In [8]:
recipient.dloss_dcomm.size()

torch.Size([1, 128])

In [6]:
agent.comm.grad